<a href="https://colab.research.google.com/github/shahhassansh/UCF_CAP5610_ML_Assignments/blob/master/Problem_1_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Common Steps for Problems 1,2 and 3**
Below are some common steps for Problem 1,2 and 3


###**Importing Libraries**

In [0]:
import numpy as np
import math
from keras.utils import to_categorical
np.random.seed(0)
from keras.datasets import mnist

###**Loading Data**

In [0]:
[train_raw_images, train_labels], [test_raw_images, test_labels] = mnist.load_data()

###**Function for Categorizing Labels**

In [0]:
def make_labels_binary(train_labels, inp):
  return [1 if x==inp else 0 for x in train_labels]

###**Generating Categorized Labels**

In [0]:
train_labels_0 = make_labels_binary(train_labels,0)
train_labels_1 = make_labels_binary(train_labels,1)
train_labels_2 = make_labels_binary(train_labels,2)
train_labels_3 = make_labels_binary(train_labels,3)
train_labels_4 = make_labels_binary(train_labels,4)
train_labels_5 = make_labels_binary(train_labels,5)
train_labels_6 = make_labels_binary(train_labels,6)
train_labels_7 = make_labels_binary(train_labels,7)
train_labels_8 = make_labels_binary(train_labels,8)
train_labels_9 = make_labels_binary(train_labels,9)

In [0]:
test_labels_0 = make_labels_binary(test_labels,0)
test_labels_1 = make_labels_binary(test_labels,1)
test_labels_2 = make_labels_binary(test_labels,2)
test_labels_3 = make_labels_binary(test_labels,3)
test_labels_4 = make_labels_binary(test_labels,4)
test_labels_5 = make_labels_binary(test_labels,5)
test_labels_6 = make_labels_binary(test_labels,6)
test_labels_7 = make_labels_binary(test_labels,7)
test_labels_8 = make_labels_binary(test_labels,8)
test_labels_9 = make_labels_binary(test_labels,9)

###**Categorizing Labels for Problem 3**

In [0]:
train_labels_new = to_categorical(train_labels)
test_labels_new = to_categorical(test_labels)

###**Image Reshaping and Normalization**

In [0]:
train_images = train_raw_images.reshape(60000, 28*28)
train_images = train_images.astype('float32') / 255

test_images = test_raw_images.reshape(10000, 28*28)
test_images = test_images.astype('float32') / 255

###**Initial Bias and Weight Declaration**

In [0]:
bias = 0
weight = np.random.rand(28*28)

bias_cce = np.zeros((10,1))
weight_cce = np.random.rand(10,28*28)

###**Logistic Regression Function**

In [0]:
def logisticRegression(train_images, weight, bias):
  z = np.dot(train_images,weight.T)+bias
  return z

def logisticRegression_cce(train_images, weight_cce, bias_cce):
  z = np.dot(weight_cce,train_images.T)+bias_cce
  return z

###**Activation Functions (Sigmoid and Softmax)**


In [0]:
## Sigmoid Function
def sigmoid(z):
   return (1./(1+np.exp(-1*z)))
  
## Softmax Function
def softmax(z):
  s = np.sum(np.exp(z), axis =0)
  return np.exp(z)/s

In [0]:
#Converting Sigmoid into 0,1
def sigmoid_binary(act):
  for i in range(0,len(act)):
    if act[i]<0.5:
      act[i] = 0
    else:
      act[i] = 1 
  return act

###**Loss Functions (Mean Square Error, Cross Entropy, Categorical Cross Function)**
These Functions are loss functions not used for training

In [0]:
## Mean Square Error Function
def MSE(act_y,pre_y,t):
  return (np.sum(np.square(pre_y - act_y)))/t

## Cross Entropy Function
def crossEntropy(act_y, pre_y,t):
  return - (np.sum(act_y * np.log(pre_y) + (1 - act_y)* np.log(1 - pre_y))) / t

## Categorical Cross Entropy Loss Function
def categoricalCrossEntropy(act_y, pre_y, t):
  return - np.sum(act_y*np.log(pre_y))

###**Optimization Function (for MSQ, CE and CCE)**

In [0]:
#OPtimization Function for Mean Sqaure Error
def optimization_MSE(A, y, t, train_images,w,b, alpha):
  dZ = (A-y)*A*(1-A)
  dW = 2*np.dot(dZ,train_images)*(1/t)
  w = w - (alpha * dW)
  dB = 2*np.sum(dZ)
  b = b - (alpha*dB)
  return w, b

In [0]:
#Optimization Function for Cross Entropy
def optimization_CE(A,y,t,train_images,w,b,alpha):
  dZ = A-y
  dW = np.dot(dZ, train_images)*(1/t)
  w = w - (alpha * dW)
  dB = np.sum(dZ)
  b = b - (alpha*dB)
  return w, b

In [0]:
#OPtimization Function for Stochastic Categorical Cross Entropy
def optimization_CCE(A, y, t, train_images,weight_cce,bias_cce, alpha):
  y = y.T
  #print(train_images.shape)
  for a in range(0,y.shape[1]):
    dZ = np.zeros((10))
    for b in range(0,y.shape[0]): #j
      aa = A[b,a]
      xx =0
      for c in range(0,y.shape[0]): #k
        if (c != b):
          xx = xx + y[c,a]*aa
        elif (c==b):
          xx = xx + y[c,a]*(aa -1)
      dZ[b] = xx
    yy =0
    for e in range(0,dZ.shape[0]):
       weight_cce[e,:]= weight_cce[e,:] - (alpha * (dZ[e]*train_images[a]))
       bias_cce[e] = bias_cce[e] - alpha * (dZ[e]) 
  return weight_cce, bias_cce

###**Accuracy Calculation**

In [0]:
def calculateAccuracy(pre,act):
  c = 0
  t = len(pre)
  for i in range(0,len(pre)):
     if (pre[i] == act[i]):
        c = c+1
  return float(c/t)*100.0
    

##**Problem 1**

###**Mini-batch Gradient Descent Model with Mean Sqaure Error Loss Method**

In [0]:
def model_MGD(train_images, train_labels, weight, bias, alpha, batchsize, epoch):
  t = len(train_images)
  x = math.ceil(t/batchsize)
  start=0
  end= start+batchsize
  for a in range(1,epoch):
    for k in range(0,x):
      z = logisticRegression(train_images[start:end], weight, bias)
      act = sigmoid(z)
      act_f = sigmoid_binary(act)
      weight,bias = optimization_MSE(act_f, train_labels[start:end], 1, train_images[start:end],weight,bias, alpha)
    start = end
    end = end+batchsize
    if (end > t):
      end = t
    acc = calculateAccuracy(act_f, train_labels[start:end])
    print("epoch: ", a, "Accuracy:", acc)
  return weight,bias

###**Traning for Category 0**

In [0]:
w0,b0 = model_MGD(train_images, train_labels_0, weight, bias, 0.001, 256, 100)

epoch:  1 Accuracy: 89.84375
epoch:  2 Accuracy: 89.84375
epoch:  3 Accuracy: 90.234375
epoch:  4 Accuracy: 92.578125
epoch:  5 Accuracy: 89.453125
epoch:  6 Accuracy: 91.015625
epoch:  7 Accuracy: 90.625
epoch:  8 Accuracy: 90.625
epoch:  9 Accuracy: 89.84375
epoch:  10 Accuracy: 91.015625
epoch:  11 Accuracy: 88.671875
epoch:  12 Accuracy: 92.1875
epoch:  13 Accuracy: 89.84375
epoch:  14 Accuracy: 89.84375
epoch:  15 Accuracy: 90.625
epoch:  16 Accuracy: 88.28125
epoch:  17 Accuracy: 91.015625
epoch:  18 Accuracy: 89.84375
epoch:  19 Accuracy: 89.0625
epoch:  20 Accuracy: 85.9375
epoch:  21 Accuracy: 88.671875
epoch:  22 Accuracy: 90.234375
epoch:  23 Accuracy: 90.625
epoch:  24 Accuracy: 91.796875
epoch:  25 Accuracy: 90.234375
epoch:  26 Accuracy: 89.84375
epoch:  27 Accuracy: 86.71875
epoch:  28 Accuracy: 90.234375
epoch:  29 Accuracy: 90.625
epoch:  30 Accuracy: 88.671875
epoch:  31 Accuracy: 87.5
epoch:  32 Accuracy: 89.453125
epoch:  33 Accuracy: 93.359375
epoch:  34 Accuracy: 

###**Test Data Accuracy Calculation for Category 0**

In [0]:
z_0=logisticRegression(test_images, w0, b0)
pre_0 = sigmoid_binary(sigmoid(z_0))
print("Testing Accuracy:",calculateAccuracy(pre_0,test_labels_0))

Testing Accuracy: 90.25


###**Training for Category 1**

In [0]:
w1,b1 = model_MGD(train_images, train_labels_1, weight, bias, 0.001, 256, 10)

epoch:  1 Accuracy: 86.71875
epoch:  2 Accuracy: 89.84375
epoch:  3 Accuracy: 90.234375
epoch:  4 Accuracy: 87.5
epoch:  5 Accuracy: 90.625
epoch:  6 Accuracy: 89.84375
epoch:  7 Accuracy: 90.234375
epoch:  8 Accuracy: 90.625
epoch:  9 Accuracy: 88.28125


###**Test Data Accuracy Calculation for Category 1**

In [0]:
z_1=logisticRegression(test_images, w1, b1)
pre_1 = sigmoid_binary(sigmoid(z_1))
print("Testing Accuracy:",calculateAccuracy(pre_1,test_labels_1))

Testing Accuracy: 88.63


###**Training for Category 2**

In [0]:
w2,b2 = model_MGD(train_images, train_labels_2, weight, bias, 0.001, 256, 10)

epoch:  1 Accuracy: 89.84375
epoch:  2 Accuracy: 89.0625
epoch:  3 Accuracy: 91.796875
epoch:  4 Accuracy: 90.234375
epoch:  5 Accuracy: 92.1875
epoch:  6 Accuracy: 87.890625
epoch:  7 Accuracy: 89.84375
epoch:  8 Accuracy: 91.40625
epoch:  9 Accuracy: 88.671875


###**Test Data Accuracy Calculation for Category 2**

In [0]:
z_2=logisticRegression(test_images, w2, b2)
pre_2 = sigmoid_binary(sigmoid(z_2))
print("Testing Accuracy:",calculateAccuracy(pre_2,test_labels_2))

Testing Accuracy: 89.69


###**Training for Category 3**

In [0]:
w3,b3 = model_MGD(train_images, train_labels_3, weight, bias, 0.001, 256, 10)

epoch:  1 Accuracy: 91.40625
epoch:  2 Accuracy: 92.1875
epoch:  3 Accuracy: 91.015625
epoch:  4 Accuracy: 90.625
epoch:  5 Accuracy: 90.234375
epoch:  6 Accuracy: 89.84375
epoch:  7 Accuracy: 89.84375
epoch:  8 Accuracy: 88.28125
epoch:  9 Accuracy: 91.40625


###**Test Data Accuracy Calculation for Category 3**

In [0]:
z_3=logisticRegression(test_images, w3, b3)
pre_3 = sigmoid_binary(sigmoid(z_3))
print("Testing Accuracy:",calculateAccuracy(pre_3,test_labels_3))

Testing Accuracy: 89.88000000000001


###**Training for Category 4**

In [0]:
w4,b4 = model_MGD(train_images, train_labels_4, weight, bias, 0.001, 256, 10)

epoch:  1 Accuracy: 88.28125
epoch:  2 Accuracy: 88.671875
epoch:  3 Accuracy: 89.0625
epoch:  4 Accuracy: 86.328125
epoch:  5 Accuracy: 92.1875
epoch:  6 Accuracy: 87.890625
epoch:  7 Accuracy: 89.453125
epoch:  8 Accuracy: 86.71875
epoch:  9 Accuracy: 88.28125


###**Test Data Accuracy Calculation for Category 4** 

In [0]:
z_4=logisticRegression(test_images, w4, b4)
pre_4 = sigmoid_binary(sigmoid(z_4))
print("Testing Accuracy:",calculateAccuracy(pre_4,test_labels_4))

Testing Accuracy: 90.14


##**Problem 2**


###**Mini-batch Gradient Descent Model with Cross Entropy Loss Method**

In [0]:
def model_MGD_CE(train_images, train_labels, weight, bias, alpha, batchsize, epoch):
  t = len(train_images)
  x = math.ceil(t/batchsize)
  start=0
  end= start+batchsize
  for a in range(1,epoch):
    for k in range(0,x):
      z = logisticRegression(train_images[start:end], weight, bias)
      act = sigmoid(z)
      act_f = sigmoid_binary(act)
      weight,bias = optimization_CE(act_f, train_labels[start:end], 1, train_images[start:end],weight,bias, alpha)
    start = end
    end = end+batchsize
    if (end > t):
      end = t
    acc = calculateAccuracy(act_f, train_labels[start:end])
    #print("epoch: ", a, "Accuracy:", acc)
  return weight,bias

###**Training for Category 5**

In [0]:
w5,b5 = model_MGD_CE(train_images, train_labels_5, weight, bias, 0.001, 256, 10)

###**Test Data Accuracy Calculation for Category 5**

In [0]:
z_5=logisticRegression(test_images, w5, b5)
pre_5 = sigmoid_binary(sigmoid(z_5))
print("Testing Accuracy:",calculateAccuracy(pre_5,test_labels_5))

Testing Accuracy: 93.47999999999999


###**Training for Category 6**

In [0]:
w6,b6 = model_MGD_CE(train_images, train_labels_6, weight, bias, 0.001, 256, 100)

###**Test Data Accuracy Calculation for Category 6**

In [0]:
z_6=logisticRegression(test_images, w6, b6)
pre_6 = sigmoid_binary(sigmoid(z_6))
print("Testing Accuracy:",calculateAccuracy(pre_6,test_labels_6))

Testing Accuracy: 97.28999999999999


###** Training for Category 7**

In [0]:
w7,b7 = model_MGD_CE(train_images, train_labels_7, weight, bias, 0.001, 256, 100)

###**Test Data Accuracy for Category 7**

In [0]:
z_7=logisticRegression(test_images, w7, b7)
pre_7 = sigmoid_binary(sigmoid(z_7))
print("Testing Accuracy:",calculateAccuracy(pre_7,test_labels_7))

Testing Accuracy: 97.23


###** Training for Category 8**


In [0]:
w8,b8 = model_MGD_CE(train_images, train_labels_8, weight, bias, 0.001, 256, 100)

###**Test Data Accuracy for Category 8**


In [0]:
z_8=logisticRegression(test_images, w8, b8)
pre_8 = sigmoid_binary(sigmoid(z_8))
print("Testing Accuracy:",calculateAccuracy(pre_8,test_labels_8))

Testing Accuracy: 93.31


###**Training for Category 9**

In [0]:
w9,b9 = model_MGD_CE(train_images, train_labels_9, weight, bias, 0.001, 256, 100)

###**Test Data Accuracy for Category 9**

In [0]:
z_9=logisticRegression(test_images, w9, b9)
pre_9 = sigmoid_binary(sigmoid(z_9))
print("Testing Accuracy:",calculateAccuracy(pre_9,test_labels_9))

Testing Accuracy: 94.64


##** Problem 3**

###**Minibatch Stochastic Gradient Descent Model with Categorical Cross Entropy Loss Method**

In [0]:
def model_MGD_CCE(train_images, train_labels, weight_cce, bias_cce, alpha, batchsize, epoch):
  t = len(train_images)
  x = math.ceil(t/batchsize)
  start=0
  end= start+batchsize
  for a in range(1,epoch):
    for k in range(0,x):
      z = logisticRegression_cce(train_images[start:end], weight_cce, bias_cce)
      #print(z.shape)
      act_f = softmax(z)
      #print(act_f.shape)
      weight,bias = optimization_CCE(act_f, train_labels_new[start:end], 1, train_images[start:end],weight_cce,bias_cce, alpha)
    start = end
    end = end+batchsize
    if (end > t):
      end = t
    #acc = calculateAccuracy(act_f, train_labels[start:end])
    #print("epoch: ", a, "Accuracy:", acc)
  return weight,bias

###**Training**

In [0]:
w,b = model_MGD_CCE(train_images, train_labels, weight_cce, bias_cce, 0.001, 256, 10)

###**Test Data Accuracy Calculation**
My Test Accuracy is 85.57 % when epoch size is 10.

In [0]:
zz=logisticRegression_cce(test_images, w, b)
#print(zz.shape)
sof = softmax(zz)
#print(sof.shape)
prep = np.argmax(sof, axis = 0)
#print(prep.shape)
print("Testing Accuracy:",calculateAccuracy(prep,test_labels))

Testing Accuracy: 85.57000000000001
